**Script que genera SCI y CCI de la última imágen disponible en el catálogo de GEE.**

SCI: Snow Cover Index

CCI: Cloud Cover Index

Este script tiene el proposito de automatizar el proceso de exportación del promedio mensual de SCI y CCI. 

To automate you need to sign-in using a service account
1. Create or Select a Google Cloud Project
2. Enable the project for 'Earth Engine API'
3. Create a service Account
    3.1 Grant role: Earth Engine Resource Writer (Avoiding Manage role to limit risk)
4. Create keys for service account (json file). Store this file securely
5. Activate the service account for GEE here (https://signup.earthengine.google.com/#!/service_accounts)
6. test sign in from python.

In [3]:
# libraries
import ee
import geemap
from datetime import datetime, timedelta
import pytz
import json
import pprint
import pathlib
from time import sleep
from yaspin import yaspin
import os

In [1]:
# Custom libraries
import nieve_sequia_auto as nsa

In [4]:
# Constants
SNOW_SERVICE_USER = 'odes-nieve@ee-proyectosequiateleamb.iam.gserviceaccount.com'
SNOW_SERVICE_CREDENTIALS_FILE = 'credentials/ee-proyectosequiateleamb-22dbf24d5c96.json'
SNOW_REGIONS_ASSET_PATH = 'users/proyectosequiateleamb/Regiones/DPA_regiones_nacional'
SNOW_EXPORT_TO = 'toAsset'
SNOW_ASSETS_PATH = 'projects/ee-proyectosequiateleamb/assets/nieve/raster_sci_cci'
SNOW_DRIVE_PATH = None
GEE_PATHPREFIX = 'projects/earthengine-legacy/assets/'
#SAVED_IMAGES_PATH = 'projects/ee-chompitest/assets/snow/Raster_SCI_CCI'
MODIS_MIN_MONTH = '2000-03'
UTC_TZ=pytz.timezone('UTC')
# Temporary test constants
#SNOW_ASSETS_PATH = 'projects/ee-chompitest/assets/snow/Raster_SCI_CCI'
#SNOW_SERVICE_USER = 'odes-nieve@ee-proyectosequiateleamb.iam.gserviceaccount.com'
#SNOW_SERVICE_CREDENTIALS_FILE = 'credentials/ee-proyectosequiateleamb-22dbf24d5c96.json'
#SNOW_MONTHS_TO_EXPORT = ['2022-12-01', '2022-11-01', '2022-10-01', '2022-09-01']


In [5]:
# Use this if you want to export specific months
# SNOW_MONTHS_TO_EXPORT: None
SNOW_MONTHS_TO_EXPORT= ["2022-12-01", "2022-11-01", "2022-10-01"]

In [6]:
# Replace Credentials with your service account
credentials = ee.ServiceAccountCredentials(SNOW_SERVICE_USER, SNOW_SERVICE_CREDENTIALS_FILE )
ee.Initialize(credentials)    

In [7]:
# Get feature collection for DPA territorio Nacional if found
if nsa.check_asset_exists(SNOW_REGIONS_ASSET_PATH, "TABLE"):
    ee_territorio_nacional = ee.FeatureCollection(SNOW_REGIONS_ASSET_PATH)


Searching for: users/proyectosequiateleamb/Regiones/DPA_regiones_nacional
projects/earthengine-legacy/assets/users/proyectosequiateleamb/Regiones/DPA_regiones_nacional:TABLE
projects/earthengine-legacy/assets/users/proyectosequiateleamb/Regiones/regiones_DPA:TABLE
Found 2 assets
Asset successfully found: True


In [8]:
# Get MODIS image Collection and take date from the last image.
# Remove images from current month to avoid incomplete months
# Check if previous month has all the images for the month
# Remove the month if not complete

ee_MODIS_collection = (ee.ImageCollection('MODIS/006/MOD10A1'))
ee_MODIS_collection = ee_MODIS_collection.filterDate(MODIS_MIN_MONTH, nsa.current_year_month())
ee_MODIS_collection = nsa.rm_incomplete_months_from_ic(ee_MODIS_collection)

Last complete month: 2022-12


In [9]:
# Get distinct months in MODIS collection
MODIS_distinct_months = nsa.get_ic_distinct_months(ee_MODIS_collection)


Total images in collection: 8273
Distinct months in collection: 273
first month: 2000-03-01
last month: 2022-12-01


In [10]:
# Get list of months of images already saved 
# TODO: Need to add option to search by asset type
saved_assets=nsa.get_asset_list(SNOW_ASSETS_PATH, "IMAGE")
saved_assets_months = [date[-7:]+'-01' for date in saved_assets]
saved_assets_months.sort(reverse=True)
print(f"Total images saved in folder: {len(saved_assets_months)}")
if len(saved_assets_months)>0:
    print(f"first month saved: {saved_assets_months[-1]}")
    print(f"last month saved: {saved_assets_months[0]}")
# mising_months = [date for date in MODIS_distinct_months if date not in saved_assets_months]

projects/ee-proyectosequiateleamb/assets/nieve/raster_sci_cci/MOD10A1_SCI_CCI_2022-12:IMAGE
Total images saved in folder: 1
first month saved: 2022-12-01
last month saved: 2022-12-01


In [20]:
# Save images of months in SNOW_MONTHS_TO_EXPORT otherwise save
# the last available month in MODIS
if SNOW_MONTHS_TO_EXPORT == None:
    try: 
        last_available = MODIS_distinct_months[0]
        months_to_save=[last_available]
    except:
        months_to_save=[]
else:
    months_to_save=SNOW_MONTHS_TO_EXPORT

# Check if months to save have already been saved
months_already_saved = [month for month in months_to_save if month in saved_assets_months]
months_to_save = [month for month in months_to_save if month not in months_already_saved]

if len(months_already_saved) >= 1:
    print(f"Months already saved: {months_already_saved}")

if len(months_to_save)==0:
    print(f"No new months to save. Exiting script")
else:
    print(f"Months to save: {months_to_save}")

Months already saved: ['2022-12-01']
Months to save: ['2022-11-01', '2022-10-01']


In [14]:

# Calculate and select SCI (snow) and CCI (cloud) bands 
ee_snow_cloud_collection = ee_MODIS_collection.map(nsa.snow_cloud_mask).select('SCI', 'CCI');


In [21]:
# Calculate mean for last month in collection
ee_monthly_snow_cloud_collection = nsa.imagecollection_monthly_mean(months_to_save, ee_snow_cloud_collection)


Processing: 2022-11-01 - 2022-12-01
Processing: 2022-10-01 - 2022-11-01


In [22]:
collection_length = ee_monthly_snow_cloud_collection.size().getInfo()
print(collection_length)

2


In [62]:
for month in months_to_save:
    print(type(month))
    print(month)
    ee_image_date = ee.Date(month)
    ee_image = ee_monthly_snow_cloud_collection.filterDate(month).first()
    #ee_image = ee_monthly_snow_cloud_collection.filter(
    #    ee.Filter.eq('system:time_start', ee.String('2022-11')))
    # ee_image = ee_monthly_snow_cloud_collection.first()
    image_date = ee_image.get('system:time_start').getInfo()
    print(image_date)
    #print(ee_image.getInfo())

<class 'str'>
2022-11-01
2022-11
<class 'str'>
2022-10-01
2022-10


In [26]:
# Sort and get last image only in case there's more than one month processed
ee_monthly_snow_cloud_collection = ee_monthly_snow_cloud_collection.sort(
        prop='system:time_start', 
        opt_ascending=False)

ee_last_snow_cloud_image = ee_monthly_snow_cloud_collection.first()
last_image_info = ee_last_snow_cloud_image.getInfo()
pprint.pprint(last_image_info)

NameError: name 'ee_monthly_snow_cloud_collection' is not defined

In [36]:
# Exportar última imagen de la coleccion al Asset GEE
property = ee_last_snow_cloud_image.get('system:time_start').getInfo()
image_name = 'MOD10A1_SCI_CCI_' + property

if SNOW_EXPORT_TO == 'toAsset':
  task = ee.batch.Export.image.toAsset(**{
    'image': ee_last_snow_cloud_image,
    'description': image_name,
    'assetId': pathlib.Path(SNOW_ASSETS_PATH, image_name).as_posix(),
    'scale': 500,
    'region': ee_territorio_nacional.geometry(),
  })
elif SNOW_EXPORT_TO == 'toDrive':
  task_config = {
        'image':ee_last_snow_cloud_image,
        'description': image_name,
        'scale': 500,
        'region':ee_territorio_nacional.geometry(),
        'maxPixels': 1E8,
        'folder': 'Raster_SCI_CCI'
    }
    
  task = ee.batch.Export.image.toDrive(**task_config)
else:
  pass # Do Nothing


In [ ]:
task.start()

In [ ]:
#check status for 5 min
nsa.check_gee_task_status(task, wait_time=300)

In [84]:
status = task.status()
pprint.pprint(status)
print(datetime.fromtimestamp(status['start_timestamp_ms']/1000, UTC_TZ))

⠦ status: RUNNING {'attempt': 1,
 'batch_eecu_usage_seconds': 4109.58056640625,
 'creation_timestamp_ms': 1673037906658,
 'description': 'MOD10A1_SCI_CCI_2022-12',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/ee-proyectosequiateleamb/assets/nieve/raster_sci_cci/MOD10A1_SCI_CCI_2022-12'],
 'id': '5LJZCV4WXMI6JHIZPBCCWID3',
 'name': 'projects/earthengine-legacy/operations/5LJZCV4WXMI6JHIZPBCCWID3',
 'start_timestamp_ms': 1673037921549,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1673043113854}
2023-01-06 20:45:21.549000+00:00
⠧ status: RUNNING 

⠏ status: RUNNING 

In [3]:
import os

In [4]:
def set_snow_variable (var):
    try: 
        globals()[var]
        try: 
            print(os.environ[var])
            globals()[var] = os.environ.get(var)
            return globals()[var]
        except:
            print(f'Using default value: {var}={globals()[var]}')
            return globals()[var]   
    except:
        print(f"Unknown variable")
        return None


In [1]:

import nieve_sequia_auto as nsa
nsa.set_snow_variable('SNOW_EXPORT_TO')

Using default value: SNOW_EXPORT_TO=toAsset


'toAsset'

In [57]:

def check_docker_secret(var):
    
    docker_secrets_path = "/var/run/secrets"
    var_path=pathlib.Path(var)
    docker_var_path = pathlib.Path(docker_secrets_path,var)   

    if len(var_path.parts) > 1 and var_path.exists():
            return var_path.as_posix()

    elif docker_var_path.exists():
            return docker_var_path.as_posix()
    else:
        print(f"file not found {var}")
        return None


In [58]:
config_file = nsa.set_script_config_var('SNOW_SERVICE_CREDENTIALS_FILE')
check_docker_secret(config_file)

Using default value: SNOW_SERVICE_CREDENTIALS_FILE=credentials/ee-proyectosequiateleamb-22dbf24d5c96.json


'credentials/ee-proyectosequiateleamb-22dbf24d5c96.json'

In [51]:
var = nsa.set_script_config_var('SNOW_SERVICE_CREDENTIALS_FILE')

docker_secrets_path = "/var/run/secrets"
var_path=pathlib.Path(var)
print(var_path.as_posix())
print(f"is dir {var_path.parts=}")
print(f"exists as is {var_path.exists()}")
docker_var_path = pathlib.Path(docker_secrets_path,var)   
print(docker_var_path.as_posix())
print(f"exists as secret: {docker_var_path.exists()}")
if var_path.parts == 1 and var_path.exists():
        print(var_path.as_posix())

elif docker_var_path.exists():
        print(docker_var_path.as_posix())
else:
    print(f"file not found {var}")
    

Using default value: SNOW_SERVICE_CREDENTIALS_FILE=credentials/ee-proyectosequiateleamb-22dbf24d5c96.json
credentials/ee-proyectosequiateleamb-22dbf24d5c96.json
is dir False
exists as is True
/var/run/secrets/credentials/ee-proyectosequiateleamb-22dbf24d5c96.json
exists as secret: False
file not found credentials/ee-proyectosequiateleamb-22dbf24d5c96.json


In [52]:
var_path = pathlib.Path("/something.json")
len(var_path.parts)

2